Based on [this](https://deliveroo.engineering/2018/12/07/monte-carlo-power-analysis.html) post from Deliveroo engineering blog.

In [1]:
import numpy as np
import pandas as pd
import altair as alt

from scipy.stats import norm, binom, mannwhitneyu
from statsmodels.stats.weightstats import ttest_ind


# t-test

In [2]:
# Sample data would be actual data measured over a fixed period of time prior to our 
# experiment. For illustration purposes here we have generated data from a normal 
# distribution.
sample_mean = 21.50
sample_sd = 12.91
sample_data = norm.rvs(loc=sample_mean, scale=sample_sd, size=20000)

sample_sizes = range(250, 20000 + 1, 250) # Sample sizes we will test over
alpha = 0.05 # Our fixed alpha
sims = 20 # The number of simulations we will run per sample size
# The minimum relative effect we will test for (3%). We could try multiple relative
# effect is we are not sure what our minimum relative effect should be
relative_effect = 1.03 
alternative = "two-sided" # Is the alternative one-sided or two-sided 

power_dist = np.empty((len(sample_sizes), 2))
for i in range(0, len(sample_sizes)): 
    N = sample_sizes[i]
    
    control_data = sample_data[0:N]
    # Multiply the control data by the relative effect, this will shift the distribution
    # of the variant left or right depending on the direction of the relative effect
    variant_data = control_data * relative_effect 
    
    significance_results = []
    for j in range(0, sims):
        # Randomly allocate the sample data to the control and variant
        rv = binom.rvs(1, 0.5, size=N) 
        control_sample = control_data[rv == True] 
        variant_sample = variant_data[rv == False]
        
        # Use Welch's t-test, make no assumptions on tests for equal variances
        test_result = ttest_ind(control_sample, variant_sample, 
                                alternative=alternative, usevar='unequal')
        
        # Test for significance
        significance_results.append(test_result[1] <= alpha) 
    # The power is the number of times we have a significant result 
    # as we are assuming the alternative hypothesis is true
    power_dist[i,] = [N, np.mean(significance_results)] 

In [3]:
power_dist = pd.DataFrame(power_dist, columns=['sample_size', 'power'])

source = power_dist

tbase = alt.Chart().encode(
    x=alt.X('sample_size', axis=alt.Axis(title='Sample size')),
    y=alt.Y('power', axis=alt.Axis(title='Power'))
)

hline = alt.Chart().mark_rule().encode(
    y=alt.Y('a:Q', axis=alt.Axis(title='')),

)

alt.layer(
    tbase.mark_point(),
    tbase.transform_loess('sample_size', 'power').mark_line(),
    hline,
    data=source
).transform_calculate(a="0.8")

alt.LayerChart(...)

# t and U test

In [4]:
# Sample data would be actual data measured over a fixed period of time prior to our 
# experiment. For illustration purposes here we have generated data from a normal 
# distribution.
sample_mean = 21.50
sample_sd = 12.91
sample_data = norm.rvs(loc=sample_mean, scale=sample_sd, size=20000)

sample_sizes = range(250, 20000 + 1, 250) # Sample sizes we will test over
alpha = 0.05 # Our fixed alpha
sims = 20 # The number of simulations we will run per sample size
# The minimum relative effect we will test for (3%). We could try multiple relative
# effect is we are not sure what our minimum relative effect should be
relative_effect = 1.03 
alternative = "two-sided" # Is the alternative one-sided or two-sided 

power_dist = np.empty((len(sample_sizes), 3))
for i in range(0, len(sample_sizes)): 
    N = sample_sizes[i]
    
    control_data = sample_data[0:N]
    # Multiply the control data by the relative effect, this will shift the distribution
    # of the variant left or right depending on the direction of the relative effect
    variant_data = control_data * relative_effect 
    
    significance_tresults = []
    significance_uresults = []
    for j in range(0, sims):
        # Randomly allocate the sample data to the control and variant
        rv = binom.rvs(1, 0.5, size=N) 
        control_sample = control_data[rv == True] 
        variant_sample = variant_data[rv == False]
        
        # Use Welch's t-test, make no assumptions on tests for equal variances
        ttest_result = ttest_ind(control_sample, variant_sample, 
                                alternative=alternative, usevar='unequal')
        # Use Mann-Whitney U-test
        utest_result = mannwhitneyu(control_sample, variant_sample, 
                                   alternative=alternative)
        
        # Test for significance
        significance_tresults.append(ttest_result[1] <= alpha)
        significance_uresults.append(utest_result[1] <= alpha)
        
    # The power is the number of times we have a significant result 
    # as we are assuming the alternative hypothesis is true
    power_dist[i,] = [N, np.mean(significance_tresults), np.mean(significance_uresults)] 

In [5]:
power_dist = pd.DataFrame(power_dist, columns=['sample_size', 'tpower', 'upower'])
power_dist = power_dist.melt(id_vars='sample_size', value_vars=['tpower', 'upower'], var_name='test', value_name='power')
labels = {'tpower':'MC Simulation t-test',
            'upower': 'MC Simulation MWW U-test'}
power_dist['test'].replace(labels, inplace=True)

In [6]:
power_dist.head()

,sample_size,test,power
0,250.0,MC Simulation t-test,0.00
1,500.0,MC Simulation t-test,0.05
2,750.0,MC Simulation t-test,0.05
3,1000.0,MC Simulation t-test,0.10
4,1250.0,MC Simulation t-test,0.20


In [7]:
dots = alt.Chart(power_dist).mark_point().encode(
    x=alt.X('sample_size', axis=alt.Axis(title='Sample size')),
    y=alt.Y('power', axis=alt.Axis(title='Power')),
    color=alt.Color('test', legend=alt.Legend(title=""))
)

hline = alt.Chart(power_dist).mark_rule().encode(
    y=alt.Y('a:Q', axis=alt.Axis(title='')),
).transform_calculate(a='0.8')

dots + dots.transform_loess('sample_size', 'power', groupby=['test']).mark_line() + hline

alt.LayerChart(...)

# Program

In [ ]:
import numpy as np
import pandas as pd
import altair as alt

from scipy.stats import norm, binom, mannwhitneyu
from statsmodels.stats.weightstats import ttest_ind

In [10]:
def get_power_dist(data,
                   sample_min,
                   sample_max,
                   stepsize,
                   relative_effect=1.03,
                   sims=20,
                   alpha=0.05,
                   alternative_t='two-sided',
                   alternative_u='two-sided',
                   tests=['t', 'u']):
    """
    Calculates power for each sample size and each specified test.
    Code is adapted version of https://deliveroo.engineering/2018/12/07/monte-carlo-power-analysis.html

    Parameters
    -----------
    sample_min : integer (required)
        minimum sample size for which power is calculated
    sample_max : integer (required)
        maximum sample size for which power is calculated
    stepsize : integer (required)
        number of sample sized skipped between each calculation
    relative_effect : float (optional)
        relative effect size of variant, default is 3 percent
    sims : integer (optional)
        number of simulations per sample size, default is 20
    alpha : float (optional)
        probability of Type I error, default is 5 percent
    alternative_t : string (optional)
        type of t-test performed, either 'two-sided' (default), 'larger', or 'smaller'
    alternative_u : string (optional)
        type of Mann-Whitney-Wilkinson U-test performed, either 'two-sided' (default), 'more', or 'less'
    tests : list of strings (optional)
        test results plotted, either ['t', 'u'] (default), ['t'], or ['u']


    Returns
    --------
    dataframe : a dataframe containing sample size and associated power levels for each test
    """ 
    sample_sizes = range(sample_min, sample_max + 1, stepsize)

    power_dist = np.empty((len(sample_sizes), 3))
    for i in range(0, len(sample_sizes)): 
        N = sample_sizes[i]
        control_data = data[0:N]
        variant_data = control_data * relative_effect 

        significance_tresults = []
        significance_uresults = []
        for j in range(0, sims):   
            # Randomly allocate the sample data to the control and variant
            rv = binom.rvs(1, 0.5, size=N) 
            control_sample = control_data[rv == True] 
            variant_sample = variant_data[rv == False]
            
            # Calculate Welch's t-test and Mann-Whitney U-test
            ttest_result = ttest_ind(control_sample, variant_sample, 
                                     alternative=alternative_t,
                                     usevar='unequal')
            utest_result = mannwhitneyu(control_sample, variant_sample, 
                                        alternative=alternative_u)
            
            # Test for significance and calculate power
            significance_tresults.append(ttest_result[1] <= alpha)
            tpower = np.mean(significance_tresults)
            significance_uresults.append(utest_result[1] <= alpha)
            upower = np.mean(significance_uresults)

        # Store results for sample size i
        power_dist[i,] = [N, tpower, upower] 
        
    # Convert to dataframe and keep results for selected tests
    power_dist = pd.DataFrame(power_dist, columns=['sample_size', 't', 'u'])
    power_dist = power_dist.melt(id_vars='sample_size',
                                 value_vars=['t', 'u'], 
                                 var_name='test',
                                 value_name='power'
                                )
    power_dist = power_dist[power_dist['test'].isin(tests)]
    labels = {'t':'MC Simulation t-test', 'u': 'MC Simulation MWW U-test'}
    power_dist['test'].replace(labels, inplace=True)

    return power_dist


def plot_power_graph(data,
                     x='sample_size',
                     y='power',
                     color='test',
                     hline_pos='0.8'):
    """
    Creates a standardised power graph
    
    Parameters
    ----------
    data : dataframe (required)
        name of dataframe
    x : string (optional)
        name of x coordinate (sample size), default is 'sample_size'
    y : string (optional)
        name of y coordinate (power), default is 'power'
    color : string (optional)
        name of variable for color encoding, default is 'test'
    hline_pos : str of float (optional)
        position of horizontal line to indicate target power level, default is '0.8'
    
    Returns
    -------
    Altair plot: A plot showing level of power for each sample size for each test.
    """
    dots = alt.Chart(data).mark_point().encode(
        x=alt.X(x, axis=alt.Axis(title='Sample size')),
        y=alt.Y(y, axis=alt.Axis(title='Power')),
        color=alt.Color(color, legend=alt.Legend(title=""))
    )
    
    loess = dots.transform_loess(x, y, groupby=[color]).mark_line()
    
    hline = alt.Chart(data).mark_rule(color='red').encode(
        y=alt.Y('a:Q', axis=alt.Axis(title='')),
    ).transform_calculate(a=hline_pos)

    return dots + loess + hline

In [12]:
newdata = norm.rvs(loc=20, scale=10, size=12000)
data = get_power_dist(newdata, 250, 12000, 250)
plot_power_graph(data)

alt.LayerChart(...)